# Kategorizálási feladat a modell használatával
A lenti kód és CLI részletek tartalmazzák a szükséges kódokat a modell kiértékelsének futtatásához

In [ ]:
echo "Read the following Wikipedia article and decide which category it belongs to: History, Science, Art, Literature, Technology, General Information. Article text: <ARTICLE_TEXT> Answer with only one word, the correct category." 
| build/bin/llama-cli -m models/mistral-7b-v0.1.Q4_K_M.gguf --chat-template chatml

In [1]:
import subprocess
import csv
import time
import re

# Fájlok elérési útvonala
input_file = "AllCombined.csv"
output_file = "Categorized_Articles.csv"
model_path = "models/mistral-7b-v0.1.Q4_K_M.gguf"
limit = 100  # Csak az első 100 cikket dolgozzuk fel

# Elfogadható kategóriák
valid_categories = {"History", "Science", "Art", "Literature", "Technology", "General Information"}

# 🔹 Indítsuk el az llama-cli-t interaktív módban
process = subprocess.Popen(
    [
        "build/bin/llama-cli", "-m", model_path, "-cnv",
        "-p", "You will categorize Wikipedia articles. Only respond with one word: History, Science, Art, Literature, Technology, or General Information. Do not explain.",
        "--reverse-prompt", "<|im_end|>"
    ],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    bufsize=1
)

# Várjunk kicsit, hogy betöltődjön
time.sleep(3)

# 🔹 Kimeneti CSV fájl inicializálása
with open(output_file, "w", newline="", encoding="utf-8") as out_csv:
    writer = csv.writer(out_csv)
    writer.writerow(["title", "category", "content"])  # Fejléc beírása

    # Bemeneti fájl beolvasása
    with open(input_file, "r", encoding="utf-8") as in_csv:
        reader = csv.reader(in_csv)
        next(reader)  # Fejléc kihagyása

        count = 0
        for row in reader:
            if count >= limit:
                break

            title, content = row[0], row[1][:200]  # Rövidített tartalom

            # 🔹 Prompt generálása
            prompt = f"Article: {content}"

            # Küldjük a kérdést a modellnek
            try:
                process.stdin.write(prompt + "\n")
                process.stdin.flush()
            except BrokenPipeError:
                print("❌ Hiba: A llama-cli folyamat megszakadt!")
                break

            # Várjunk a válaszra
            time.sleep(2.5)  # Kicsit hosszabb várakozás, hogy biztosan feldolgozza a választ

            # 🔹 Modell válaszának beolvasása
            reads = 0
            category = "Unknown"
            while reads < 4 : 
                response = process.stdout.readline().strip()
                print(f"📢 Nyers válasz: {response}")

                # 🔹 Extra karakterek és felesleges formázások eltávolítása
                response = re.sub(r"^> ", "", response)  # ">" eltávolítása a válasz elejéről
                response = response.replace("<|im_end|>", "").strip()  # Speciális tokenek eltávolítása

                # Ha nincs válasz, vagy rossz válasz jött, akkor "Unknown"
                if response in valid_categories:
                    category = response
                    break
                reads += 1

            # Eredmény mentése CSV-be
            writer.writerow([title, category, content])

            count += 1

# 🔹 Folyamat lezárása
try:
    process.stdin.close()
    process.wait()
except BrokenPipeError:
    print("❌ Hiba a folyamat lezárásánál.")

print("✅ Kategorizálás sikeresen befejezve!")

FileNotFoundError: [WinError 2] A rendszer nem találja a megadott fájlt